In [1]:
!pip install requests

see : https://metmuseum.github.io/

In [1]:
import requests
image_url="http://data.closertovaneyck.be/ec2/tiles/00-10-VIS-HI-AT/17/0_0.jpg"

save_path="dataset/391_230.jpg"
img_data = requests.get(image_url).content
with open(save_path, 'wb') as handler:
    handler.write(img_data)
print(img_data)

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\x02\x02\x03\x03\x03\x03\x04\x03\x03\x04\x05\x08\x05\x05\x04\x04\x05\n\x07\x07\x06\x08\x0c\n\x0c\x0c\x0b\n\x0b\x0b\r\x0e\x12\x10\r\x0e\x11\x0e\x0b\x0b\x10\x16\x10\x11\x13\x14\x15\x15\x15\x0c\x0f\x17\x18\x16\x14\x18\x12\x14\x15\x14\xff\xdb\x00C\x01\x03\x04\x04\x05\x04\x05\t\x05\x05\t\x14\r\x0b\r\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\xff\xc0\x00\x11\x08\x01\x00\x01\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\

In [6]:
import numpy as np
import requests
from PIL import Image
import io



def find_y_boundery():
    x=0
    found_picture=False
    while(True):
        image_url=f"http://data.closertovaneyck.be/ec2/tiles/00-10-VIS-HI-AT/17/{x}_100.jpg"
        request=requests.get(image_url)
        img_data =request.content
        image_numpy=np.asarray(Image.open(io.BytesIO(img_data)))
        is_nothing=np.all(image_numpy==image_numpy.mean())
        if is_nothing:
            if found_picture:#si c'est le bord
                return xmin,x-1
        else:
            if not(found_picture):#first time we found something
                xmin=x#
                found_picture=True
            
        x=x+1

def find_x_boundery():
    x=0
    found_picture=False
    while(True):
        image_url=f"http://data.closertovaneyck.be/ec2/tiles/00-10-VIS-HI-AT/17/100_{x}.jpg"
        request=requests.get(image_url)
        img_data =request.content
        image_numpy=np.asarray(Image.open(io.BytesIO(img_data)))
        is_nothing=np.all(image_numpy==image_numpy.mean())
        if is_nothing:
            if found_picture:#si c'est le bord
                return xmin,x-1
        else:
            if not(found_picture):#first time we found something
                xmin=x#
                found_picture=True
            
        x=x+1

xmin,xmax=find_x_boundery()
ymin,ymax=find_y_boundery()
print(xmax,ymax)#should take approximatively 1 min
print(xmin,ymin)

231 392
16 12


In [2]:
xmax,ymax=231,292
xmin,ymin=16,12
number_of_files=(xmax)*(ymax)
number_of_files

67452

In [2]:
#test the optimum number of cores for getting the datas
import requests

import os.path
import time

def save_img_from_url(x,y):
    
    image_url=f"http://data.closertovaneyck.be/ec2/tiles/00-10-VIS-HI-AT/17/{y}_{x}.jpg"
    save_path=f"dataset/{y}_{x}.jpg"
    if False:#os.path.isfile(save_path):
        pass
    else:
        request=requests.get(image_url)
        img_data =request.content
        
        with open(save_path, 'wb') as handler:
            handler.write(img_data)

number_of_files=200#xmax*ymax
list_x=[i%xmax for i in range(number_of_files)]
list_y=[i//xmax for i in range(number_of_files)]
for i in range(100,3,-5):
    

    t=time.time()

    with ThreadPoolExecutor(max_workers=i) as pool:
        response_list = list(pool.map(save_img_from_url,list_x,list_y))

    print(f"for {i} workers, time elapsed = {time.time()-t} for {number_of_files} files")

for 100 workers, time elapsed = 4.440263986587524 for 200 files
for 95 workers, time elapsed = 1.3584389686584473 for 200 files
for 90 workers, time elapsed = 1.6002652645111084 for 200 files


KeyboardInterrupt: 

In [3]:
#get the datas
import random
import requests


def save_img_from_url(x,y,resolution=17):
    
    image_url=f"http://data.closertovaneyck.be/ec2/tiles/00-10-VIS-HI-AT/{resolution}/{y}_{x}.jpg"
    save_path=f"dataset/{y}_{x}.jpg"
    if os.path.isfile(save_path):
        pass
    else:
        if x>=xmin and y>=ymin:#if the image is not empty
            request=requests.get(image_url)
            img_data =request.content
            if random.randint(0,1000)==0:#print a message with 0.1% probability
                print(x,y)
            with open(save_path, 'wb') as handler:
                handler.write(img_data)
      

number_of_files=xmax*ymax
list_x=[i%xmax for i in range(number_of_files)]
list_y=[i//xmax for i in range(number_of_files)]


In [11]:
ymin

12

In [4]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=100) as pool:
    response_list = list(pool.map(save_img_from_url,list_x,list_y))



121 229
204 229
198 232


KeyboardInterrupt: 